## District Math and Reading Test Performance Analysis

- On average, across the district students perform better on reading than they do math.
- Bottom performing schools have a disparity between math and reading average scores than the top performing schools
- The top performing school does not spend the most per student. In fact, schools spending the most per student performed worse on average than those on the lower end of district per student spending. 
- Medium size schools performed the best over all across reading and math tests. Large schools had the greatest disparity between students passing math tests vs students passing reading tests.
- Charter schools greatly outperformed district schools.


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
school_data_to_load = "Resources/PyCity_School_Data.csv"
student_data_to_load = "Resources/PyCity_Student_Data.csv"

In [ ]:
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)


## Initializing Data

In [ ]:
school_data = school_data.rename(columns={"school_name" : "School Name", "type" : "Type", "size" : "Size", "budget" : "Budget"})
school_data

In [ ]:
student_data = student_data.rename(columns={"student_name" : "Student Name", "gender" : "Gender", "grade" : "Grade",
                                           "school_name" : "School Name", "reading_score" : "Reading Score", "math_score" : "Math Score"})
student_data.head()

In [ ]:
school_complete = pd.merge(student_data, school_data, how="left", on=["School Name", "School Name"])

In [ ]:
school_complete = school_complete.rename(columns={"student_name" : "Student Name", "gender" : "Gender", "grade" : "Grade", "school_name" : "School Name",
                                                 "reading_score" : "Reading Score", "math_score" : "Math Score", "type" : "Type", "size" : "Size",
                                                 "budget" : "Budget"})
school_complete.head()


## District Summary

In [ ]:
#Calculate number of schools in the district (used .nunique() function to return count of unique school names).
district_schools = school_complete["School Name"].nunique()

#calculate total number of students in the district.
district_students = school_complete["Student Name"].count()

#cacluate total distruct budget (sum of each school's budget).
district_budget = school_data["Budget"].sum()

#Calculate the average math score.
avg_math_score = school_complete["Math Score"].mean()

#Calculate the average reading score.
avg_reading_score = school_complete["Reading Score"].mean()

#----------------------------------------------------------------------
#Calculate the overall average score

avg_overall = ((avg_math_score + avg_reading_score) / 2)

#---------------------------------------------------------------------
#Calculate the percentage of students with a passing math score.

#total count of students with math score of 70 or greater
passing_math_total = school_complete.loc[school_complete["Math Score"] >= 70].count()["Student ID"]


#Use count of studnets with score of 70% or greater and total count of students overall to calculate the percentage
#and store as a variable
passing_math_pct = (passing_math_total / district_students)*100

#----------------------------------------------------------------------------
#calculate the percentage of students witha  passing reading score.

#total count of students with reading score of 70 or greater.
passing_reading_total = school_complete.loc[school_complete["Reading Score"] >= 70].count()["Student ID"]

#Use count of studnets with score of 70% or greater and total count of students overall to calculate the percentage
#and store as a variable
passing_reading_pct = (passing_reading_total / district_students)*100

#----------------------------------------------------------------------------------
#Use variales above to create a district summary data frame.
district_summary = pd.DataFrame({"Total Schools" : [district_schools],
                                 "Total Students" : [district_students],
                                 "Total Budget" : [district_budget],
                                 "Average Math Score" : [avg_math_score],
                                 "Average Reading Score" : [avg_reading_score],
                                 "% Passing Math" : [passing_math_pct],
                                 "% Passing Reading" : [passing_reading_pct],
                                 "% Overall Passing Rate" : [avg_overall]})


#Format data
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}".format)
district_summary["% Overall Passing Rate"] = district_summary["% Overall Passing Rate"].map("{:.2f}".format)


district_summary


## School Summary

In [ ]:
#Using the school_data as starting point, set index to the "School Name" column.
school_summary = school_data.set_index(["School Name"])

#Find the budget per student for each school.
budget_per_student = school_summary["Budget"] / school_summary["Size"]

#Find average math and reading scores for each school.
sch_avg_math_score = school_complete.groupby("School Name").mean()["Math Score"]
sch_avg_reading_score = school_complete.groupby("School Name").mean()["Reading Score"]

#Find number of studnets at each school with score over 70 for math and reading (respectively)
sch_passing_math_count = school_complete.loc[school_complete["Math Score"] >= 70].groupby("School Name").count()["Student ID"]
sch_passing_reading_count = school_complete.loc[school_complete["Reading Score"] >= 70].groupby("School Name").count()["Student ID"]

#Calculate the percentage of students with passing reading and math scores for each scool.
sch_passing_math_pct = (sch_passing_math_count / school_summary["Size"]) * 100
sch_passing_reading_pct = (sch_passing_reading_count / school_summary["Size"]) * 100

#Calculate the percentage of students with passing scores overall at each school.
sch_overall_passign_rate = (sch_passing_math_pct + sch_passing_reading_pct) / 2

# Assign cacluated values above to new columns and add to the school summary data frame.
school_summary["Per Student Budget"] = budget_per_student
school_summary["Average Math Score"] = sch_avg_math_score
school_summary["Average Reading Score"] = sch_avg_reading_score
school_summary["% Passing Math"] = sch_passing_math_pct
school_summary["% Passing Reading"] = sch_passing_reading_pct
school_summary["% Overall Passing Rate"] = sch_overall_passign_rate

#Reoganize columns.

school_summary = school_summary.rename(columns={"Size" : "Total Students", "Type" : "School Type"})
school_summary = school_summary[[ "School Type", "Total Students", "Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", 
                   "% Passing Reading", "% Overall Passing Rate"]]

#Format data
school_summary_formatted = school_summary.copy()


school_summary_formatted["Total Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary_formatted["Budget"] = school_summary["Budget"].map("${:,.2f}".format)
school_summary_formatted["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary_formatted["Average Math Score"] = school_summary["Average Math Score"].map("{:,.2f}".format)
school_summary_formatted["Average Reading Score"] = school_summary["Average Reading Score"].map("{:,.2f}".format)
school_summary_formatted["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}".format)
school_summary_formatted["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}".format)
school_summary_formatted["% Overall Passing Rate"] = school_summary["% Overall Passing Rate"].map("{:.2f}".format)

school_summary_formatted



## Top Performing Schools (by passing rate)

In [ ]:
#Sort school summary by % overall passing rate in decending order
top_performing_schools = school_summary_formatted.sort_values(["% Overall Passing Rate"], ascending = False)

top_performing_schools.head(5)



## Bottom Performing Schools (by passing rate)

In [ ]:
#Sort school summary by % overall passing rate in ascending order
bottom_performing_schools = school_summary_formatted.sort_values(["% Overall Passing Rate"], ascending = True)

bottom_performing_schools.head(5)

## Math Scores by Grade

In [ ]:
#Create series containing the average math scores for students in each grade.
ninth_grade_math = school_complete.loc[school_complete["Grade"] == "9th"].groupby("School Name").mean()["Math Score"]
tenth_grade_math = school_complete.loc[school_complete["Grade"] == "10th"].groupby("School Name").mean()["Math Score"]
eleventh_grade_math = school_complete.loc[school_complete["Grade"] == "11th"].groupby("School Name").mean()["Math Score"]
twelfth_grade_math = school_complete.loc[school_complete["Grade"] == "12th"].groupby("School Name").mean()["Math Score"]

#Create a data frame using the series established above.
math_score_by_grade_summary = pd.DataFrame({"9th" : ninth_grade_math, "10th" : tenth_grade_math,
                                            "11th" : eleventh_grade_math, "12th" : twelfth_grade_math})


#Format data
math_score_by_grade_summary["9th"] = math_score_by_grade_summary["9th"].map("{:,.2f}".format)
math_score_by_grade_summary["10th"] = math_score_by_grade_summary["10th"].map("{:,.2f}".format)
math_score_by_grade_summary["11th"] = math_score_by_grade_summary["11th"].map("{:,.2f}".format)
math_score_by_grade_summary["12th"] = math_score_by_grade_summary["12th"].map("{:,.2f}".format)

math_score_by_grade_summary


## Reading Scores by Grade

In [ ]:
#Create series containing the average reading scores for students in each grade.
ninth_grade_reading = school_complete.loc[school_complete["Grade"] == "9th"].groupby("School Name").mean()["Reading Score"]
tenth_grade_reading = school_complete.loc[school_complete["Grade"] == "10th"].groupby("School Name").mean()["Reading Score"]
eleventh_grade_reading = school_complete.loc[school_complete["Grade"] == "11th"].groupby("School Name").mean()["Reading Score"]
twelfth_grade_reading = school_complete.loc[school_complete["Grade"] == "12th"].groupby("School Name").mean()["Reading Score"]

#Create a data frame using the series established above.
reading_score_by_grade_summary = pd.DataFrame({"9th" : ninth_grade_reading, "10th" : tenth_grade_reading,
                                            "11th" : eleventh_grade_reading, "12th" : twelfth_grade_reading})
#Format data
reading_score_by_grade_summary["9th"] = reading_score_by_grade_summary["9th"].map("{:,.2f}".format)
reading_score_by_grade_summary["10th"] = reading_score_by_grade_summary["10th"].map("{:,.2f}".format)
reading_score_by_grade_summary["11th"] = reading_score_by_grade_summary["11th"].map("{:,.2f}".format)
reading_score_by_grade_summary["12th"] = reading_score_by_grade_summary["12th"].map("{:,.2f}".format)

reading_score_by_grade_summary


## Scores by School Spending

In [ ]:

scores_by_school_spending = school_summary.copy()

#Define bins to organize the data by.
spending_bins = [0, 585, 615, 645, 675]

#Assign names for the bins to be used as labels
spending_group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#Use the cut function to insert a new column containing describing the bin the record belongs to.
scores_by_school_spending["Spending Per Student Range"] = pd.cut(scores_by_school_spending["Per Student Budget"], spending_bins, labels=spending_group_names)

#Create series of the average scores, % passing math and reading, and % overall passing, grouping by the bins
#defined above.
spend_math_avg = scores_by_school_spending.groupby("Spending Per Student Range").mean()["Average Math Score"]
spend_read_avg = scores_by_school_spending.groupby("Spending Per Student Range").mean()["Average Reading Score"]
spend_pct_pass_math = scores_by_school_spending.groupby("Spending Per Student Range").mean()["% Passing Math"]
spend_pct_pass_read = scores_by_school_spending.groupby("Spending Per Student Range").mean()["% Passing Reading"]
spend_pct_overall_pass = scores_by_school_spending.groupby("Spending Per Student Range").mean()["% Overall Passing Rate"]

#Create a data frame using the series established above.
scores_by_school_spending = pd.DataFrame({"Average Math Score" : spend_math_avg, "Average Reading Score" : spend_read_avg,
                                         "% Passing Math" : spend_pct_pass_math, "% Passing Reading" : spend_pct_pass_read,
                                         "% Overall Passing Rate" : spend_pct_overall_pass})

#Format the data
scores_by_school_spending_formatted = scores_by_school_spending.copy()

scores_by_school_spending_formatted["Average Math Score"] = scores_by_school_spending_formatted["Average Math Score"].map("{:,.2f}".format)
scores_by_school_spending_formatted["Average Reading Score"] = scores_by_school_spending_formatted["Average Reading Score"].map("{:,.2f}".format)
scores_by_school_spending_formatted["% Passing Math"] = scores_by_school_spending_formatted["% Passing Math"].map("{:.2f}".format)
scores_by_school_spending_formatted["% Passing Reading"] = scores_by_school_spending_formatted["% Passing Reading"].map("{:.2f}".format)
scores_by_school_spending_formatted["% Overall Passing Rate"] = scores_by_school_spending_formatted["% Overall Passing Rate"].map("{:.2f}".format)

scores_by_school_spending_formatted


## Scores  by School Size

In [ ]:

scores_by_school_size = school_summary.copy()

#Define bins to organize the data by.
size_bins = [0, 1000, 2000, 5000]

#Assign names for the bins to be used as labels
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Use the cut function to insert a new column containing describing the bin the record belongs to.
scores_by_school_size["School Size Range"] = pd.cut(scores_by_school_size["Total Students"], size_bins, labels=size_group_names)

#Create series of the average scores, % passing math and reading, and % overall passing, grouping by the bins
#defined above.
size_math_avg = scores_by_school_size.groupby("School Size Range").mean()["Average Math Score"]
size_read_avg = scores_by_school_size.groupby("School Size Range").mean()["Average Reading Score"]
size_pct_pass_math = scores_by_school_size.groupby("School Size Range").mean()["% Passing Math"]
size_pct_pass_read = scores_by_school_size.groupby("School Size Range").mean()["% Passing Reading"]
size_pct_overall_pass = scores_by_school_size.groupby("School Size Range").mean()["% Overall Passing Rate"]

#Create a data frame using the series established above.
scores_by_school_size = pd.DataFrame({"Average Math Score" : size_math_avg, "Average Reading Score" : size_read_avg,
                                         "% Passing Math" : size_pct_pass_math, "% Passing Reading" : size_pct_pass_read,
                                         "% Overall Passing Rate" : size_pct_overall_pass})

#Format the data.
scores_by_school_size_formatted = scores_by_school_size.copy()

scores_by_school_size_formatted["Average Math Score"] = scores_by_school_size_formatted["Average Math Score"].map("{:,.2f}".format)
scores_by_school_size_formatted["Average Reading Score"] = scores_by_school_size_formatted["Average Reading Score"].map("{:,.2f}".format)
scores_by_school_size_formatted["% Passing Math"] = scores_by_school_size_formatted["% Passing Math"].map("{:.2f}".format)
scores_by_school_size_formatted["% Passing Reading"] = scores_by_school_size_formatted["% Passing Reading"].map("{:.2f}".format)
scores_by_school_size_formatted["% Overall Passing Rate"] = scores_by_school_size_formatted["% Overall Passing Rate"].map("{:.2f}".format)

scores_by_school_size_formatted


## Scores by School Type

In [ ]:

scores_by_school_type= school_summary.copy()

#Create series of the average scores, % passing math and reading, and % overall passing, grouping by school type.
type_math_avg = scores_by_school_type.groupby("School Type").mean()["Average Math Score"]
type_read_avg = scores_by_school_type.groupby("School Type").mean()["Average Reading Score"]
type_pct_pass_math = scores_by_school_type.groupby("School Type").mean()["% Passing Math"]
type_pct_pass_read = scores_by_school_type.groupby("School Type").mean()["% Passing Reading"]
type_pct_overall_pass = scores_by_school_type.groupby("School Type").mean()["% Overall Passing Rate"]

#Create a data frame using the series established above.
scores_by_school_type = pd.DataFrame({"Average Math Score" : type_math_avg, "Average Reading Score" : type_read_avg,
                                         "% Passing Math" : type_pct_pass_math, "% Passing Reading" : type_pct_pass_read,
                                         "% Overall Passing Rate" : type_pct_overall_pass})

#Format the data
scores_by_school_type_formatted = scores_by_school_type.copy()

scores_by_school_type_formatted["Average Math Score"] = scores_by_school_type_formatted["Average Math Score"].map("{:,.2f}".format)
scores_by_school_type_formatted["Average Reading Score"] = scores_by_school_type_formatted["Average Reading Score"].map("{:,.2f}".format)
scores_by_school_type_formatted["% Passing Math"] = scores_by_school_type_formatted["% Passing Math"].map("{:.2f}".format)
scores_by_school_type_formatted["% Passing Reading"] = scores_by_school_type_formatted["% Passing Reading"].map("{:.2f}".format)
scores_by_school_type_formatted["% Overall Passing Rate"] = scores_by_school_type_formatted["% Overall Passing Rate"].map("{:.2f}".format)

scores_by_school_type_formatted